In [1]:
import numpy as np
import psytrack as psy
import pickle 
import copy
import os
import datetime

%load_ext autoreload
%autoreload 2

import helperFns as mf

curPath = os.path.abspath(os.getcwd())

In [25]:
## Code to extract GLM weight trajectories from behavioral data for all mice. Some elements adapted from Roy et al, 2021. 

IDs = ('GS027', 'GS028', 'GS029', 'GS030', 'GS037', 'GS040', 'JC025', 'JC028', 'JC029', 'JC039', 'JC044', 'JC047', 'JC048', 'JC052', 'JC057', 'JC059', 'JC061', 'JC062', 'JC067')

untilTesting = False # We're removing some of the sessions at the end to make sure that the mice "end" at similar accuracies

saveDat = True

weights = {'bias': 1,
           's_high': 1,
           's_low': 1}

K = np.sum([weights[i] for i in weights.keys()])

hyper= {'sigInit': 2**4.,      # Set to a single, large value for all weights. Will not be optimized further.
        'sigma': [2**-4.]*K,   # Each weight will have it's own sigma optimized, but all are initialized the same
        'sigDay': 2**-4.}      # If NONE, then ignore session boundaries

optList = ['sigma','sigDay']

for ID in IDs:

    print('Starting ' + ID)
    
    keyword = "training"
    stc = 'training'
    
    cutoff = 60
    numSessions = None
    accCutoff = 0
    tD = mf.getD(ID, keyword = keyword, cutoff = cutoff, accCutoff = accCutoff, sessionCutoff = numSessions, txt = None, untilTesting = untilTesting)
    
    tD2 = copy.deepcopy(tD)

    hyp, evd, wMode, hess_info = psy.hyperOpt(tD2, hyper, weights, optList)

    psytrack = {
        'wMode': wMode,
        'weights': weights,
        'hyp': hyp,
    }

    tD['psytrack'] = psytrack

    today = datetime.datetime.now()
    date_time = today.strftime("%y%m%d")

    tmp = 'new_data/' + date_time + '/with_bias_learning/'
   
    if not os.path.isdir(tmp):
        os.makedirs(tmp)

    if saveDat:
        saveName = tmp + "{}_trainingDataBias.pkl".format(ID)
        with open(saveName, 'wb') as f:
            pickle.dump(tD, f)


Starting GS027
Starting GS028
Starting GS029
Starting GS030
Starting GS037
Starting GS040
Starting JC025
Starting JC028
Starting JC029
Starting JC039
Starting JC044
Starting JC047
Starting JC048
Starting JC052
Starting JC057
Starting JC059
Starting JC061
Starting JC062
Starting JC067
